In [ ]:
import os
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pytz

connection = sqlite3.connect(os.getenv("SQLITE_DB_FILE"))
cursor = connection.cursor()

query = """
SELECT activity.timestamp, video.title, video.length, channel.name AS channel_name, category.name AS category_name
FROM activity
JOIN video ON activity.video_id = video.id
JOIN channel ON activity.channel_id = channel.id
JOIN category ON channel.category_id = category.id"""

df = pd.read_sql_query(query, connection)
connection.close()

# Convert the timestamp column to a datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Convert UTC timestamp to Finnish local time
utc_zone = pytz.timezone('UTC')
finland_zone = pytz.timezone('Europe/Helsinki')
df['timestamp'] = df['timestamp'].apply(lambda x: x.replace(tzinfo=utc_zone).astimezone(finland_zone))

# Extract the hour from the timestamp
df['hour'] = df['timestamp'].dt.hour

plt.style.use('dark_background')
# Create a count plot to visualize video views per hour
plt.figure(figsize=(11, 4))
sns.countplot(data=df, x='hour', color='#1E90FF')
plt.title('Views by Hour of the Day (Finnish Local Time)')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Views')
plt.show()